In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/HIV_Resistance/GSE46599'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Genome-wide analysis of interferon-stimulated genes in primary cells and immortalized cell lines"
!Series_summary	"Analysis of interferon-stimulated genes (ISGs) in various primary cells and immortalized cell lines, following type 1 interferon (IFN) treatment. Some cell types become resistant to HIV-1 infection following type 1 interferon treatment (such as macrophages, THP-1, PMA-THP-1, U87-MG cells and to a lesser extent, primary CD4+ T cells) while others either become only partially resistant (e.g., HT1080, PMA-U937) or remain permissive (e.g., CEM, CEM-SS, Jurkat T cell lines and U937); for more information see (Goujon and Malim, Journal of Virology 2010) and (Goujon and Schaller et al., Retrovirology 2013). We hypothesized that the anti-HIV-1 ISGs are differentially induced and expressed in restrictive cells compared to permissive cells and performed a whole genome analysis following type 1 IFN treatment in cell types exhibiting different HI

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Part 1: Check if gene expression data is available
is_gene_available = True

# Part 2: Define trait, age, gender row keys and conversion functions

# 2.1 Data Availability
trait_row = 4
age_row = None
gender_row = None

# 2.3 Data Type Conversion
def convert_trait(value):
    value = value.split(':')[-1].strip()  # Extract value part
    if value == 'resistant':
        return 1
    elif value == 'permissive' or value == 'untreated':
        return 0
    elif value == 'partially resistant':
        return 0.5  # or any defined logic consistent with the study requirements
    else:
        return None

def convert_age(value):
    return None  # Not applicable since age data is not available

def convert_gender(value):
    return None  # Not applicable since gender data is not available

# Part 3: Save Metadata
save_cohort_info('GSE46599', './preprocessed/HIV_Resistance/cohort_info.json', is_gene_available, trait_row is not None)

# Part 4: Clinical Feature Extraction
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'HIV_Resistance', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/HIV_Resistance/trait_data/GSE46599.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


{'GSM1133032': [0], 'GSM1133033': [0], 'GSM1133034': [0], 'GSM1133035': [0], 'GSM1133036': [0], 'GSM1133037': [0], 'GSM1133038': [0], 'GSM1133039': [0], 'GSM1133040': [0.5], 'GSM1133041': [0.5], 'GSM1133042': [0], 'GSM1133043': [0], 'GSM1133044': [0], 'GSM1133045': [0], 'GSM1133046': [0], 'GSM1133047': [0], 'GSM1133048': [1], 'GSM1133049': [1], 'GSM1133050': [0], 'GSM1133051': [0], 'GSM1133052': [0.5], 'GSM1133053': [0.5], 'GSM1133054': [0], 'GSM1133055': [0], 'GSM1133056': [1], 'GSM1133057': [1], 'GSM1133058': [1], 'GSM1133059': [0], 'GSM1133060': [0], 'GSM1133061': [0], 'GSM1133062': [1], 'GSM1133063': [1], 'GSM1133064': [1], 'GSM1133065': [0], 'GSM1133066': [0], 'GSM1133067': [0], 'GSM1133068': [1], 'GSM1133069': [1], 'GSM1133070': [0], 'GSM1133071': [0], 'GSM1133072': [1], 'GSM1133073': [1], 'GSM1133074': [0], 'GSM1133075': [0], 'GSM1133076': [0], 'GSM1133077': [0], 'GSM1133078': [0], 'GSM1133079': [0]}


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row ids for the following step.
print(gene_data.index[:20])


Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651236',
       'ILMN_1651238', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259',
       'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278',
       'ILMN_1651281', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
requires_gene_mapping = True


### Step 5: Gene Annotation (Conditional)

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the soft file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCAT

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify relevant keys in gene annotation data
identifier_key = 'ID'
gene_symbol_key = 'Symbol'

# 2. Get the dataframe storing the mapping between probe IDs and genes 
mapping_df = get_gene_mapping(gene_annotation, identifier_key, gene_symbol_key)

# 3. Apply the mapping to obtain the gene expression dataframe
gene_data = apply_gene_mapping(gene_data, mapping_df)

# The resulting gene_data is now ready for further analysis.


### Step 7: Data Normalization and Merging

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
gene_csv_path = './preprocessed/HIV_Resistance/gene_data/GSE46599.csv'
os.makedirs(os.path.dirname(gene_csv_path), exist_ok=True)  # Ensure directory exists
normalized_gene_data.to_csv(gene_csv_path)

# 2. Merge the clinical and genetic data with the 'geo_merge_clinical_genetic_data' function from the library.
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, normalized_gene_data)

# 3. Determine whether the trait and some demographic attributes in the data is severely biased, and remove biased attributes.
trait_biased, unbiased_merged_data = judge_and_remove_biased_features(merged_data, 'HIV_Resistance')

# 4. Save the cohort information.
json_path = './preprocessed/HIV_Resistance/cohort_info.json'
os.makedirs(os.path.dirname(json_path), exist_ok=True)  # Ensure directory exists
save_cohort_info('GSE46599', json_path, True, True, trait_biased, unbiased_merged_data)

if not trait_biased:
    # 5. If the trait is not severely biased, save the merged data to a csv file.
    csv_path = './preprocessed/HIV_Resistance/GSE46599.csv'
    os.makedirs(os.path.dirname(csv_path), exist_ok=True)  # Ensure directory exists
    unbiased_merged_data.to_csv(csv_path)


Quartiles for 'HIV_Resistance':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.625
Min: 0.0
Max: 1.0
The distribution of the feature 'HIV_Resistance' in this dataset is fine.

